In [1]:
def cross(A, B):
    return [a+b for a in A for b in B]

rows = 'ABCDEFGHI'
cols = '123456789'
digits = cols
squares = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123','456','789')]
           )
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

In [2]:
def test():
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')

In [3]:
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

In [4]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

In [5]:
def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

In [6]:
def display(values):
    #"Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else ''))
                      for c in cols)
        if r in 'CF': print(line)
    print()

In [7]:
def solve(grid): return search(parse_grid(grid))

def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares): 
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    return some(search(assign(values.copy(), s, d)) 
            for d in values[s])

def some(seq):
    "Return some element of seq that is true."
    for e in seq:
        if e: return e
    return False

In [13]:
hard1= '.....6....59.....82....8....45........3........6..3.54...325..6..................'
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid = '040201060000000000905000307000000000507080104010000090001000600000705000608904503'
t = '040201050000000000905000307000000000507080104010000090001000500000705000608904503'

In [14]:
parse_grid(grid)

{'A1': '7',
 'A2': '4',
 'A3': '3',
 'A4': '2',
 'A5': '9',
 'A6': '1',
 'A7': '8',
 'A8': '6',
 'A9': '5',
 'B1': '1',
 'B2': '8',
 'B3': '6',
 'B4': '5',
 'B5': '3',
 'B6': '7',
 'B7': '9',
 'B8': '4',
 'B9': '2',
 'C1': '9',
 'C2': '2',
 'C3': '5',
 'C4': '8',
 'C5': '4',
 'C6': '6',
 'C7': '3',
 'C8': '1',
 'C9': '7',
 'D1': '3',
 'D2': '6',
 'D3': '4',
 'D4': '1',
 'D5': '7',
 'D6': '9',
 'D7': '2',
 'D8': '5',
 'D9': '8',
 'E1': '5',
 'E2': '9',
 'E3': '7',
 'E4': '6',
 'E5': '8',
 'E6': '2',
 'E7': '1',
 'E8': '3',
 'E9': '4',
 'F1': '8',
 'F2': '1',
 'F3': '2',
 'F4': '4',
 'F5': '5',
 'F6': '3',
 'F7': '7',
 'F8': '9',
 'F9': '6',
 'G1': '4',
 'G2': '5',
 'G3': '1',
 'G4': '3',
 'G5': '2',
 'G6': '8',
 'G7': '6',
 'G8': '7',
 'G9': '9',
 'H1': '2',
 'H2': '3',
 'H3': '9',
 'H4': '7',
 'H5': '6',
 'H6': '5',
 'H7': '4',
 'H8': '8',
 'H9': '1',
 'I1': '6',
 'I2': '7',
 'I3': '8',
 'I4': '9',
 'I5': '1',
 'I6': '4',
 'I7': '5',
 'I8': '2',
 'I9': '3'}